In [1]:
%env PYTHONPATH=E:/Polito/Tesi/rl_portfolio_management

!python src/test_agent.py

env: PYTHONPATH=E:/Polito/Tesi/rl_portfolio_management

[*********************100%***********************]  1 of 1 completed



Traceback (most recent call last):
  File "e:\PoliTo\Tesi\rl_portfolio_management\src\test_agent.py", line 36, in <module>
    main()
  File "e:\PoliTo\Tesi\rl_portfolio_management\src\test_agent.py", line 23, in main
    sac = SACAgent('sac', env, seed, sac_config)
  File "E:\Polito\Tesi\rl_portfolio_management\src\agents\sac_agent.py", line 18, in __init__
    super().__init__(name, env, seed, args)
  File "E:\Polito\Tesi\rl_portfolio_management\src\agents\base_ac_agent.py", line 48, in __init__
    self.checkpoint_folder = f'./checkpoints/{args.checkpoint_folder}_{args.network_type}'
AttributeError: 'Struct' object has no attribute 'network_type'


In [1]:
import torch
from src.utils.file_utils import read_yaml_config
from src.environments.portfolio_end import PortfolioEnd
from src.models.gaussian_actor import GaussianActor
from src.models.critic import Critic
from src.utils.torch_utils import USE_CUDA, FLOAT

device = torch.device("cuda:0" if USE_CUDA else "cpu")
env_config_train = read_yaml_config('env_default_train')
env_train = PortfolioEnd(env_config_train)

obs = env_train.reset()
a,b = obs
a = torch.tensor(a, dtype=FLOAT, device=device)
b = torch.tensor(b, dtype=FLOAT, device=device)

a = torch.stack((a,a))
b = torch.stack((b,b))
print(a.shape)
ga = GaussianActor(3, 30).cuda()

c,d = ga(a,b)
c

crit = Critic(3,17,30).cuda()
e = crit(a,b,c)
print(d.shape)

[*********************100%***********************]  1 of 1 completed
torch.Size([2, 3, 16, 30])
torch.Size([2, 1])


In [5]:
import torch
from src.utils.file_utils import read_yaml_config
from src.environments.portfolio_end import PortfolioEnd

env_config_train = read_yaml_config('env_default_train')
env_train = PortfolioEnd(env_config_train)

print(-torch.prod(torch.tensor(env_train.action_space.shape)))

tensor(-17)


In [4]:
import numpy as np

a = np.empty(1)
a = np.append(a,1)
print(len([]))

0


In [57]:
from unicodedata import bidirectional
import torch
import torch.nn as nn

class LSTM(nn.Module):

    def __init__(self):
        super().__init__()
        self.lstm1 = nn.LSTM(48,16,2, batch_first=True)

        self.fc = nn.Linear(16,16)


    def forward(self, x):
        
        x1, (hn,cn) = self.lstm1(x)

        a = self.fc(x1)
        b = self.fc(hn)


        return a,b

class LSTM2(nn.Module):
    
    def __init__(self):
        super().__init__()
        self.fc = nn.Linear(16,16)

        self.lstm3 = nn.LSTM(48,16,batch_first=True)
        self.lstm4 = nn.LSTM(16,16,batch_first=True)

    def forward(self, x):

        x2, (hn,cn) = self.lstm3(x)
        x2, (hn,cn) = self.lstm4(x2)

        a2 = self.fc(x2)
        b2 = self.fc(hn)

        return a2,b2



lstm = LSTM()
lstm2 = LSTM2()

a = torch.rand(64,30,48)
#out, (hidden, cell) = lstm(a)
a1,a2 = lstm(a)
a3,a4 = lstm2(a)
print(a1.shape)
print(a2.shape)
print(a3.shape)
print(a4.shape)
print(torch.allclose(a1[:,-1,:], a2[-1,:,:]))
print(torch.allclose(a3[:,-1,:], a4.squeeze()))
print(torch.allclose(a1,a3))
print(torch.allclose(a2[-1,:,:],a4))

torch.Size([64, 30, 16])
torch.Size([2, 64, 16])
torch.Size([64, 30, 16])
torch.Size([1, 64, 16])
True
True
False
False


In [61]:
import torch
from ptflops import get_model_complexity_info

num_classes = 19

with torch.cuda.device(0):

    # define models
    model = LSTM()
    model_depthwise = LSTM2()

    # calculate complexities
    macs, params = get_model_complexity_info(model, (30,48), as_strings=False,
                                        print_per_layer_stat=False, verbose=False)
    macs_depthwise, params_depthwise = get_model_complexity_info(model_depthwise, (30,48), as_strings=False,
                                                  print_per_layer_stat=False, verbose=False)


# FLOPs ~= 2 * MACS
print(f'Discriminator - FLOPs: {2 * macs}')
print(f'Depthwise separable discriminator - FLOPs: {2 * macs_depthwise}')
print(f'Discriminator - Params: {params}')
print(f'Depthwise separable discriminator - Params: {params_depthwise}')

Discriminator - FLOPs: 419648.0
Depthwise separable discriminator - FLOPs: 419136.0
Discriminator - Params: 6672
Depthwise separable discriminator - Params: 6672


In [7]:
import numpy as np
import yfinance as yf
import datetime as dt

_start_date = '2010-01-01'
_end_date = '2010-01-11'
_date_fmt = '%Y-%m-%d'

yf.download('AAPL',
start=(dt.datetime.strptime(_start_date, _date_fmt) +
        dt.timedelta(1)).strftime(_date_fmt),
end=(dt.datetime.strptime(_end_date, _date_fmt) + dt.timedelta(1)).strftime(_date_fmt)
).drop(columns=['Adj Close'])



[*********************100%***********************]  1 of 1 completed


,Open,High,Low,Close,Volume
Date,,,,,
2010-01-04,7.622500,7.660714,7.585000,7.643214,493729600
2010-01-05,7.664286,7.699643,7.616071,7.656429,601904800
2010-01-06,7.656429,7.686786,7.526786,7.534643,552160000
2010-01-07,7.562500,7.571429,7.466071,7.520714,477131200
2010-01-08,7.510714,7.571429,7.466429,7.570714,447610800
2010-01-11,7.600000,7.607143,7.444643,7.503929,462229600


In [3]:
from src.environments.market import Market

m = Market('2010-01-01', '2010-01-09', 3, ['AAPL', 'SPY'])

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


In [6]:
print(m.tot_steps)

2
